### RAG_valid -> Reflection -> final VCP (Python data)

In [3]:
%pip install -q openai pandas scikit-learn tqdm


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ================== Imports and OpenAI client helper ==================
import os, re, csv
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
from openai import OpenAI
import getpass
from typing import Optional


def init_openai_client(api_key: Optional[str] = None, persist_env: bool = False) -> Optional[OpenAI]:
    """Initialize an OpenAI client securely.

    Behavior:
    - If api_key is provided, use it to create the client (but avoid hardcoding keys in notebooks).
    - Otherwise try the OPENAI_API_KEY environment variable.
    - If still missing, prompt the user (hidden input) to paste the key for the current session.
    - If persist_env=True, the key will be stored in os.environ for this process only (not written to disk).

    Returns an OpenAI client instance or None if no key was provided/valid.
    """
    if api_key is None:
        api_key = os.getenv("OPENAI_API_KEY")

    if not api_key:
        # Secure prompt (hidden) so the key isn't visible in notebook outputs or history
        try:
            typed = getpass.getpass("Enter your OpenAI API key (input hidden). Press Enter to skip: ")
        except Exception:
            typed = None
        if typed:
            api_key = typed.strip()

    if not api_key:
        print("No OpenAI API key provided. To run code that calls OpenAI, set the OPENAI_API_KEY environment variable or call init_openai_client(api_key=...) in the notebook (not recommended to paste keys in files).")
        return None

    if persist_env:
        os.environ["OPENAI_API_KEY"] = api_key

    try:
        client = OpenAI(api_key=api_key)
        print("OpenAI client initialized.")
        return client
    except Exception as e:
        # Fail gracefully and show a short hint
        print(f"Failed to initialize OpenAI client: {e}")
        print("Check the key or your network. Don't paste long keys into the notebook permanently.")
        return None


# Initialize OpenAI client
client = init_openai_client(api_key=os.getenv(''))



OpenAI client initialized.


In [ ]:
DATA_CSV = ""       # must have: code_before, code_after, comment, gt_label
OUT_CSV  = ""

MODEL_NAME = "gpt-4o"      # latest multimodal GPT-4 model (text + reasoning)
MAX_TOKENS = 6             # only need "YES" or "NO"


In [12]:
STRICT_PROMPT_HEADER = """[Context Section]

You are analyzing examples from Stack Overflow where a code snippet (code_before) was later edited (code_after) and accompanied by a comment (comment).
The task is to decide if the code changes were likely caused or motivated by the comment — that is, whether the comment influenced the edit.
The answer must be "YES" if the comment and the code edit are semantically or functionally related (e.g., the edit fixes, extends, or corrects something mentioned or implied by the comment).
Otherwise, the answer must be "NO".

[Conditions / Decision Rules]

You must answer YES only if all of the following are true:

The comment refers to appreciation first then suggests a change that addresses the issue that was solved by the edit.

There is a semantic or lexical overlap between what the comment discusses and what changed in the code.

The comment appears before or in a way that could trigger the change (not an approval after the fix).

Answer NO if any of the following are true:

The comment is off-topic, descriptive, or acknowledgment-only (“thanks,” “works now,” “perfectly correct,” etc.).

The comment’s topic is different from what changed in the code.

The edit fixes an issue not mentioned or not implied by the comment.

The comment occurs after the fix or merely confirms correctness like phrase (e.g., Thanks you, it works, its perfectly fine)

[Input Format]
code_before:
<original code>

code_after:
<modified code>

comment:
<text of the comment>

[Query / Task Instruction]

Compare the code_before and code_after.
Read the comment carefully and determine whether the change between the two code versions was caused or motivated by that comment.

Provide the final answer strictly as:

YES
or
NO

No explanation, reasoning, or extra words.
"""

def build_prompt(code_before: str, code_after: str, comment: str) -> str:
    return (
        STRICT_PROMPT_HEADER
        + "\ncode_before:\n"
        + str(code_before).strip()
        + "\n\ncode_after:\n"
        + str(code_after).strip()
        + "\n\ncomment:\n"
        + str(comment).strip()
        + "\n\n"
    )


In [13]:
def normalize_yesno(text: str) -> str:
    """
    Map raw model output to strict YES/NO.
    """
    if not text:
        return "NO"
    t = text.strip()
    if t.upper().startswith("Y"):
        return "YES"
    if t.upper().startswith("N"):
        return "NO"
    # fallback
    m = re.search(r"\b(YES|NO)\b", t.upper())
    return m.group(1) if m else "NO"


def ask_gpt4o(prompt: str) -> str:
    """
    Send prompt to GPT-4o and return normalized YES/NO.
    """
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=MAX_TOKENS,
        temperature=0.0,      # deterministic
    )
    text = response.choices[0].message.content.strip()
    return normalize_yesno(text)


In [14]:
df = pd.read_csv(DATA_CSV)
print("Rows:", len(df))
print("Columns:", df.columns.tolist())
df.head(2)


Rows: 217
Columns: ['Unnamed: 0', 'rag_pred', 'gt', 'PostId', 'code_before', 'code_after', 'AcceptanceDate', 'LastEditDate', 'EditorId', 'CommentDate', 'CommenterId', 'comment', 'CommentScore', 'CTMLabel', 'CotLabel', 'SO_Revisions_Link', 'verdict', 'Comment Type', 'Confidence']


Unnamed: 0 rag_pred       gt    PostId  \
0           0    valid  invalid  73672870   
1           1    valid    valid  73603795   

                                         code_before  \
0  You created batches of one element each by set...   
1  You are doing two different operations in your...   

                                          code_after           AcceptanceDate  \
0  You created batches of one element each by set...  2022-09-10 00:00:00 UTC   
1  You are doing two different operations in your...  2022-09-05 00:00:00 UTC   

                  LastEditDate  EditorId    CommentDate  CommenterId  \
0   2022-09-12 11:17:50.63 UTC  14774959  9/10/22 18:36    6859580.0   
1  2022-09-05 01:55:28.487 UTC   5854731    9/5/22 1:51    5854731.0   

                                             comment  CommentScore  CTMLabel  \
0  Just so I understand, can you show me how the ...             1         1   
1  Ok, Can you try this `test[["ex1","ex2"]] = te...             1         1   

   CotLabel                              SO_Revisions_Link  verdict  \
0         0  https://stackoverflow.com/questions/73672870/        0   
1         0  https://stackoverflow.com/questions/73603795/        1   

  Comment Type Confidence  
0     Question       0.75  
1     Solution       0.75

In [15]:
df["gt_label"] = df["gt"].apply(lambda x: "YES" if str(x).strip().upper() == "VALID" else "NO")
df[["gt_label"]].head()

gt_label
0       NO
1      YES
2      YES
3       NO
4       NO

In [16]:
df

Unnamed: 0 rag_pred       gt    PostId  \
0             0    valid  invalid  73672870   
1             1    valid    valid  73603795   
2             2    valid    valid  73694160   
3             3    valid  invalid  11597911   
4             4    valid  invalid  73736507   
..          ...      ...      ...       ...   
212         212    valid    valid  13786327   
213         213    valid    valid  73830003   
214         214    valid    valid   3244061   
215         215    valid    valid  73587450   
216         216    valid  invalid  73729998   

                                           code_before  \
0    You created batches of one element each by set...   
1    You are doing two different operations in your...   
2    You need to set ServerName to the hostname in ...   
3    The main advantages are that it makes multi-li...   
4    There is probably a more efficient way to code...   
..                                                 ...   
212  Here's a couple of suggestions:\n\nUse date_ra...   
213  Just like I commented, your variables with _re...   
214  Here's code making use of PIL and Scipy's clus...   
215  one solution would be to run it through [eval]...   
216  Try this:\n\n    l = ['Man City', 'Chelsea', '...   

                                            code_after  \
0    You created batches of one element each by set...   
1    You are doing two different operations in your...   
2    You need to set ServerName to the hostname in ...   
3    The main advantages are that it makes multi-li...   
4    There is probably a more efficient way to code...   
..                                                 ...   
212  Here's a couple of suggestions:\n\nUse date_ra...   
213  Your variables with _remainder are redundant.\...   
214  Here's code making use of Pillow and Scipy's c...   
215  one solution would be to run it through litera...   
216  Try this:\nl = ['Man City', 'Chelsea', 'Man Ci...   

              AcceptanceDate                 LastEditDate  EditorId  \
0    2022-09-10 00:00:00 UTC   2022-09-12 11:17:50.63 UTC  14774959   
1    2022-09-05 00:00:00 UTC  2022-09-05 01:55:28.487 UTC   5854731   
2    2022-09-12 00:00:00 UTC  2022-09-12 19:12:14.757 UTC    374642   
3    2012-07-22 00:00:00 UTC   2022-09-02 21:12:10.99 UTC   4518341   
4    2022-09-16 00:00:00 UTC   2022-09-16 15:33:31.75 UTC  16201558   
..                       ...                          ...       ...   
212  2012-12-15 00:00:00 UTC   2022-09-19 17:24:55.79 UTC     63550   
213  2022-09-23 00:00:00 UTC  2022-09-23 15:45:05.833 UTC   3381305   
214  2010-07-14 00:00:00 UTC  2022-09-20 01:04:21.897 UTC   1426065   
215  2022-09-02 00:00:00 UTC  2022-09-02 20:20:01.897 UTC  13592469   
216  2022-09-15 00:00:00 UTC  2022-09-15 11:48:06.343 UTC  11355926   

       CommentDate  CommenterId  \
0    9/10/22 18:36    6859580.0   
1      9/5/22 1:51    5854731.0   
2    9/12/22 20:32   19979668.0   
3     9/6/20 12:11    4972447.0   
4    9/16/22 15:34   16201558.0   
..             ...          ...   
212  10/20/13 6:17          NaN   
213  9/23/22 15:40    4831822.0   
214   5/6/19 14:18     221537.0   
215   9/2/22 20:16   15873043.0   
216  9/15/22 11:43   11355926.0   

                                               comment  CommentScore  \
0    Just so I understand, can you show me how the ...             1   
1    Ok, Can you try this `test[["ex1","ex2"]] = te...             1   
2    It worked! I changed the second line to "Serve...             1   
3                     This answer is really beautiful.             1   
4    @ouroboros1 true; I added an edit based on you...             1   
..                                                 ...           ...   
212  I noticed there is a typo in the example in th...             1   
213  @Pepinallo11 Alternatively use your previous `...             1   
214  Thanks @philshem. I believe I've modified it t...             2   
215  `eval` is potentially dangerous. Use `ast.lite.

In [17]:
preds = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    cb = str(row.get("code_before", ""))
    ca = str(row.get("code_after", ""))
    cm = str(row.get("comment", ""))
    gt = str(row.get("gt_label", "")).strip().upper()

    prompt = build_prompt(cb, ca, cm)
    pred = ask_gpt4o(prompt)

    preds.append(pred)

df["pred_label"] = preds
df.head()


100%|██████████| 217/217 [02:54<00:00,  1.24it/s]


Unnamed: 0 rag_pred       gt    PostId  \
0           0    valid  invalid  73672870   
1           1    valid    valid  73603795   
2           2    valid    valid  73694160   
3           3    valid  invalid  11597911   
4           4    valid  invalid  73736507   

                                         code_before  \
0  You created batches of one element each by set...   
1  You are doing two different operations in your...   
2  You need to set ServerName to the hostname in ...   
3  The main advantages are that it makes multi-li...   
4  There is probably a more efficient way to code...   

                                          code_after           AcceptanceDate  \
0  You created batches of one element each by set...  2022-09-10 00:00:00 UTC   
1  You are doing two different operations in your...  2022-09-05 00:00:00 UTC   
2  You need to set ServerName to the hostname in ...  2022-09-12 00:00:00 UTC   
3  The main advantages are that it makes multi-li...  2012-07-22 00:00:00 UTC   
4  There is probably a more efficient way to code...  2022-09-16 00:00:00 UTC   

                  LastEditDate  EditorId    CommentDate  ...  \
0   2022-09-12 11:17:50.63 UTC  14774959  9/10/22 18:36  ...   
1  2022-09-05 01:55:28.487 UTC   5854731    9/5/22 1:51  ...   
2  2022-09-12 19:12:14.757 UTC    374642  9/12/22 20:32  ...   
3   2022-09-02 21:12:10.99 UTC   4518341   9/6/20 12:11  ...   
4   2022-09-16 15:33:31.75 UTC  16201558  9/16/22 15:34  ...   

                                             comment CommentScore  CTMLabel  \
0  Just so I understand, can you show me how the ...            1         1   
1  Ok, Can you try this `test[["ex1","ex2"]] = te...            1         1   
2  It worked! I changed the second line to "Serve...            1         1   
3                   This answer is really beautiful.            1         1   
4  @ouroboros1 true; I added an edit based on you...            1         1   

   CotLabel                              SO_Revisions_Link verdict  \
0         0  https://stackoverflow.com/questions/73672870/       0   
1         0  https://stackoverflow.com/questions/73603795/       1   
2         1  https://stackoverflow.com/questions/73694160/       1   
3         0  https://stackoverflow.com/questions/11597911/       0   
4         1  https://stackoverflow.com/questions/73736507/       0   

   Comment Type Confidence gt_label pred_label  
0      Question       0.75       NO         NO  
1      Solution       0.75      YES        YES  
2          Flaw        0.5      YES         NO  
3          Flaw        0.5       NO         NO  
4      Solution       0.55       NO        YES  

[5 rows x 21 columns]

In [ ]:
df["pred_label"].value_counts()

pred_label
NO     109
YES    108
Name: count, dtype: int64

In [ ]:
df.to_csv("")

In [ ]:
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
cm = confusion_matrix(df["gt_label"], df["pred_label"], labels=["YES", "NO"])
print("Confusion Matrix (rows: true, columns: pred):\n")
print(pd.DataFrame(cm, index=["TRUE:YES", "TRUE:NO"], columns=["PRED:YES", "PRED:NO"]))

Confusion Matrix (rows: true, columns: pred):

          PRED:YES  PRED:NO
TRUE:YES        85       12
TRUE:NO         23       97


# 🧩 Confusion Matrix and Classification Metrics

## Step 1: Original Confusion Matrix

| Actual \ Predicted | 0 | 1 | Total |
|--------------------|---|---|--------|
| **0** | 307 | 120 | 427 |
| **1** | 46  | 97  | 143 |
| **Total** | 353 | 217 | 570 |

---

## Step 2: Re-evaluation of Predicted=1 Group (217 Samples)

| Actual \ New Predicted | 0 | 1 | Total |
|------------------------|---|---|--------|
| **0** | 97 | 23 | 120 |
| **1** | 12 | 85 | 97 |
| **Total** | 109 | 108 | 217 |

---

## Step 3: Final Combined Confusion Matrix

| Actual \ Predicted | 0 | 1 | Total |
|--------------------|---|---|--------|
| **0** | **404** | **23** | 427 |
| **1** | **58** | **85** | 143 |
| **Total** | 462 | 108 | 570 |

---

## ✅ Step 4: Evaluation Metrics

| Metric | Class 0 | Class 1 |
|---------|----------|----------|
| **Precision** | 0.874 | 0.787 |
| **Recall** | 0.946 | 0.595 |
| **F1-score** | 0.909 | 0.677 |

---

## 🔍 Summary

- **Class 0 (Invalid)** shows high recall (0.946) — the model captures most invalid cases correctly.  
- **Class 1 (Valid)** achieves decent precision (0.787) but moderate recall (0.595), indicating some valid cases are still missed.  
- Overall, the model performs more confidently on **invalid** instances than **valid** ones.

---

## 📊 Key Takeaways

- Re-evaluation improved the model's precision for both classes.  
- There remains a recall imbalance — future work could focus on improving detection of valid edits through data balancing or targeted few-shot examples.


In [ ]:
df.to_csv("", index=False)